## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avera,US,33.1940,-82.5271,72.55,71,100,10.96,overcast clouds
1,1,Rabo De Peixe,PT,37.8000,-25.5833,63.91,77,40,11.50,scattered clouds
2,2,Busselton,AU,-33.6500,115.3333,58.73,71,0,8.25,clear sky
3,3,Springfield,US,37.2153,-93.2982,49.89,60,0,9.22,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,76.96,81,59,9.89,broken clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Puerto Ayora,EC,-0.7393,-90.3518,76.96,81,59,9.89,broken clouds
29,29,Victoria,HK,22.2855,114.1577,81.18,75,0,8.01,clear sky
31,31,Sola,VU,-13.8833,167.5500,82.56,73,8,10.04,clear sky
35,35,Nabire,ID,-3.3667,135.4833,77.70,89,100,2.86,overcast clouds
40,40,Talaja,IN,21.3500,72.0500,85.62,55,0,17.43,clear sky
43,43,Georgetown,MY,5.4112,100.3354,84.15,84,20,2.30,few clouds
45,45,Eenhana,NaN,-17.4667,16.3333,83.12,38,11,4.07,few clouds
48,48,Rikitea,PF,-23.1203,-134.9692,77.67,71,7,18.32,clear sky
52,52,Saint-Joseph,RE,-21.3667,55.6167,77.38,86,61,16.20,light rain
53,53,Komsomolskiy,UZ,40.4272,71.7189,77.72,25,2,0.00,clear sky


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                220
City                   220
Country                216
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
import numpy as np

preferred_cities_df['Country'].replace('', np.nan, inplace=True)
preferred_cities_df.count()

clean_df = preferred_cities_df
clean_df.count()

City_ID                216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [56]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
29,Victoria,HK,81.18,clear sky,22.2855,114.1577,
31,Sola,VU,82.56,clear sky,-13.8833,167.5500,
35,Nabire,ID,77.70,overcast clouds,-3.3667,135.4833,
40,Talaja,IN,85.62,clear sky,21.3500,72.0500,
43,Georgetown,MY,84.15,few clouds,5.4112,100.3354,
48,Rikitea,PF,77.67,clear sky,-23.1203,-134.9692,
52,Saint-Joseph,RE,77.38,light rain,-21.3667,55.6167,
53,Komsomolskiy,UZ,77.72,clear sky,40.4272,71.7189,
54,Bambous Virieux,MU,80.22,broken clouds,-20.3428,57.7575,


In [57]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
#     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [60]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
29,Victoria,HK,81.18,clear sky,22.2855,114.1577,Mini Hotel Central
31,Sola,VU,82.56,clear sky,-13.8833,167.5500,Leumerus Bungalows
35,Nabire,ID,77.70,overcast clouds,-3.3667,135.4833,Hotel Nusantara I
40,Talaja,IN,85.62,clear sky,21.3500,72.0500,Tofik Memon
43,Georgetown,MY,84.15,few clouds,5.4112,100.3354,Cititel Penang
48,Rikitea,PF,77.67,clear sky,-23.1203,-134.9692,People ThankYou
52,Saint-Joseph,RE,77.38,light rain,-21.3667,55.6167,"""Plantation Bed and Breakfast"
53,Komsomolskiy,UZ,77.72,clear sky,40.4272,71.7189,Ikathouse
54,Bambous Virieux,MU,80.22,broken clouds,-20.3428,57.7575,Casa Tia Villa


In [61]:
hotel_df.count()

City                   202
Country                202
Max Temp               202
Current Description    202
Lat                    202
Lng                    202
Hotel Name             202
dtype: int64

In [64]:
# 7. Drop the rows where there is no Hotel Name.
# Dont need to - Just in case
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.count()



clean_hotel_df = hotel_df
clean_hotel_df
clean_hotel_df.count()

City                   202
Country                202
Max Temp               202
Current Description    202
Lat                    202
Lng                    202
Hotel Name             202
dtype: int64

In [65]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the Vacation Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [77]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

#<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Max Temp</dt><dd>76.96 °F</dd>\n<dt>Current Weather</dt><dd>broken clouds</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mini Hotel Central</dd>\n<dt>City</dt><dd>Victoria</dd>\n<dt>Country</dt><dd>HK</dd>\n<dt>Max Temp</dt><dd>81.18 °F</dd>\n<dt>Current Weather</dt><dd>clear sky</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Leumerus Bungalows</dd>\n<dt>City</dt><dd>Sola</dd>\n<dt>Country</dt><dd>VU</dd>\n<dt>Max Temp</dt><dd>82.56 °F</dd>\n<dt>Current Weather</dt><dd>clear sky</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Nusantara I</dd>\n<dt>City</dt><dd>Nabire</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Max Temp</dt><dd>77.7 °F</dd>\n<dt>Current Weather</dt><dd>overcast clouds</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Tofik Memon</dd>\n<dt>City</dt><dd>Talaja</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Max Temp</dt><dd>85.62 °F</dd>\n<dt>C

In [78]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))